In [71]:
import streamlit as st
import numpy as np
import pickle
import h5py
from datetime import datetime, timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import arch

import plotly.graph_objs as go

import yfinance as yf
from datetime import datetime, timedelta

In [72]:
%pwd

'd:\\Datamites_Training\\stockmarket_project'

In [73]:
# Function to preprocess and split data
def preprocess_data(data):
    data.reset_index(inplace=True)
    data.dropna(inplace=True)
    data['LTTS.NS100'] = data['Close'].rolling(100).mean()
    data['LTTS.NS200'] = data['Close'].rolling(200).mean()
    return data


In [74]:

def scale_data(train_data, test_data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train_data[['Close']])
    test_scaled = scaler.transform(test_data[['Close']])
    return scaler, train_scaled, test_scaled

In [75]:

def create_dataset(data, look_back=100):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)


In [76]:
# Function to train neural network models
def train_neural_network(x_train, y_train):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(x_train.shape[1],)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=0)
    return model

In [77]:
# Function to train LSTM model
def train_lstm(x_train, y_train):
    LSTM_model = Sequential()
    LSTM_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    LSTM_model.add(LSTM(units=50))
    LSTM_model.add(Dense(units=1))
    LSTM_model.compile(optimizer='adam', loss='mse')
    LSTM_model.fit(x_train, y_train, epochs=50, batch_size=32,verbose=0)
    return LSTM_model

In [78]:
# Function to save models
def save_model(model, filename):
    model.save(filename)

In [79]:
# Function to train and save Random Forest and Gradient Boosting models
def train_and_save_ensemble_models(x_train, y_train):
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(x_train, y_train)
    with open('artifacts/random_forest_regressor_model.pkl', 'wb') as f:
        pickle.dump(rf_model, f)

    gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    gb_model.fit(x_train, y_train)
    with open('artifacts/gradientboost_regressor_model.pkl', 'wb') as f:
        pickle.dump(gb_model, f)

In [80]:
# Function to make predictions
def make_predictions(model, x_test):
    return model.predict(x_test)

In [81]:
# Function to evaluate models
def evaluate_models(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    return mae, mse

In [82]:
# Function to train GARCH model
def train_garch_model(data):
    garch_model = arch.arch_model(data['Close'], vol='GARCH', p=1, q=1)
    garch_res = garch_model.fit(update_freq=5)
    garch_volatility = garch_res.conditional_volatility
    return garch_res, garch_volatility

In [83]:
# Function to plot original vs predicted prices
def plot_predictions(test_data, y_test_padded, y_pred_models, model_names=None):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_data['Date'], y=y_test_padded.flatten(), mode='lines', name='Original Price', line=dict(color='black')))
    colors = ['red', 'green', 'purple', 'darkblue']
    if model_names is None:
        model_names = ['Model 1', 'Model 2', 'Model 3', 'Model 4']
    for i, y_pred in enumerate(y_pred_models, start=1):
        fig.add_trace(go.Scatter(x=test_data['Date'].iloc[100:], y=y_pred.flatten(), mode='lines', name=model_names[i-1], line=dict(color=colors[i-1])))
    fig.update_layout(title='Original vs Predicted Prices', xaxis_title='Date', yaxis_title='Price')
    fig.show()


In [84]:
# User input for stock ticker symbol
stock_symbol = st.sidebar.text_input('Enter Stock Ticker Symbol (e.g., MSFT):')

# Date range input
start_date = st.sidebar.date_input('Select Start Date:', datetime.now() - timedelta(days=365))
end_date = st.sidebar.date_input('Select End Date:', datetime.now())

    
# Define stock data retrieval parameters
start_date = '2000-01-01'
end_date = '2024-02-01'
stock_symbol = 'LTTS.NS'

In [85]:
# Load stock data
if stock_symbol:
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    st.subheader('Stock Data')
    st.write(stock_data.head(50))  # Display first 50 rows


    # Download stock data
    data = yf.download(stock_symbol, start=start_date, end=end_date)
    data = preprocess_data(data)

    # Split data into train and test sets
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]

    scaler, train_scaled, test_scaled = scale_data(train_data, test_data)

    x_train, y_train = create_dataset(train_scaled)
    x_test, y_test = create_dataset(test_scaled)

    # Train and save neural network models
    nn_model = train_neural_network(x_train, y_train)
    save_model(nn_model, 'artifacts/NN_model.h5')

    # Train and save LSTM model
    lstm_model = train_lstm(x_train, y_train)
    save_model(lstm_model, 'artifacts/LSTM_model.h5')

    # Train and save Random Forest and Gradient Boosting models
    train_and_save_ensemble_models(x_train, y_train)

    # # GARCH model
    # garch_model = arch.arch_model(train_data['Close'], vol='GARCH', p=1, q=1)
    # garch_res = garch_model.fit(update_freq=5)
    # garch_volatility = garch_res.conditional_volatility

    # Make predictions
    nn_pred = make_predictions(nn_model, x_test)
    lstm_pred = make_predictions(lstm_model, x_test)

    with open('artifacts/random_forest_regressor_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)
   
    rf_pred = make_predictions(rf_model, x_test)

    with open('artifacts/gradientboost_regressor_model.pkl', 'rb') as f:
        gb_model = pickle.load(f)
              
    gb_pred = make_predictions(gb_model, x_test)

    # Train GARCH model
    garch_res, garch_volatility = train_garch_model(data)
        
    # Save the trained GARCH model using pickle
    with open('artifacts/garch_model.pkl', 'wb') as f:
        pickle.dump(garch_res, f)

    # # Reshape GARCH volatility for compatibility with other predictions
    # garch_volatility = garch_volatility.values.reshape(-1, 1)

    # # Repeat or interpolate garch_volatility to match the size of x_test
    # garch_volatility_resized = np.repeat(garch_volatility, len(x_test) // len(garch_volatility) + 1)[:len(x_test)]

    # # Reshape garch_volatility to match the shape of x_test
    # garch_volatility_resized = garch_volatility_resized.reshape(-1, 1)

    # # Concatenate garch_volatility with other features
    # x_test_with_garch = np.concatenate((x_test, garch_volatility_resized), axis=1)

    # # Make predictions using modified features
    # nn_pred_with_garch = make_predictions(nn_model, x_test_with_garch)
    # lstm_pred_with_garch = make_predictions(lstm_model, x_test_with_garch)
    # rf_pred_with_garch = make_predictions(rf_model, x_test_with_garch)
    # gb_pred_with_garch = make_predictions(gb_model, x_test_with_garch)

    # # Inverse transform the GARCH model predictions
    # garch_pred = scaler.inverse_transform(garch_volatility.reshape(-1, 1))

    # Inverse transform the predictions and actual values
    y_pred_models = [scaler.inverse_transform(pred.reshape(-1, 1)) for pred in [nn_pred, lstm_pred, rf_pred, gb_pred]]
    # y_pred_models_with_garch = [scaler.inverse_transform(pred.reshape(-1, 1)) for pred in [nn_pred_with_garch, lstm_pred_with_garch, rf_pred_with_garch, gb_pred_with_garch]]
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Plot predictions
    plot_predictions(test_data, np.concatenate((np.zeros((100, 1)), y_test)), y_pred_models)
    # plot_predictions(test_data, np.concatenate((np.zeros((100, 1)), y_test)), y_pred_models_with_garch, model_names=['Neural Network (with GARCH)', 'LSTM (with GARCH)', 'Random Forest (with GARCH)', 'Gradient Boosting (with GARCH)'])

    # Evaluate models
    mae_mse = [evaluate_models(y_test, y_pred) for y_pred in y_pred_models]
    # mae_mse_with_garch = [evaluate_models(y_test, y_pred) for y_pred in y_pred_models_with_garch]

    st.write('Without GARCH:')
    st.write(f'Neural Network: MAE={mae_mse[0][0]}, MSE={mae_mse[0][1]}')
    st.write(f'LSTM: MAE={mae_mse[1][0]}, MSE={mae_mse[1][1]}')
    st.write(f'Random Forest: MAE={mae_mse[2][0]}, MSE={mae_mse[2][1]}')
    st.write(f'Gradient Boosting: MAE={mae_mse[3][0]}, MSE={mae_mse[3][1]}')

    # Display GARCH model summary
    st.subheader('GARCH Model Summary')
    st.write(garch_res.summary())

# st.write('With GARCH:')
# st.write(f'Neural Network (with GARCH): MAE={mae_mse_with_garch[0][0]}, MSE={mae_mse_with_garch[0][1]}')
# st.write(f'LSTM (with GARCH): MAE={mae_mse_with_garch[1][0]}, MSE={mae_mse_with_garch[1][1]}')
# st.write(f'Random Forest (with GARCH): MAE={mae_mse_with_garch[2][0]}, MSE={mae_mse_with_garch[2][1]}')
# st.write(f'Gradient Boosting (with GARCH): MAE={mae_mse_with_garch[3][0]}, MSE={mae_mse_with_garch[3][1]}')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


c:\Users\User\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



9/9 [==============================] - 1s 18ms/step
Iteration:      5,   Func. Count:     28,   Neg. LLF: 15308.082903448008
Iteration:     10,   Func. Count:     53,   Neg. LLF: 72628.36262770054
Iteration:     15,   Func. Count:     83,   Neg. LLF: 15282.714393904096
Iteration:     20,   Func. Count:    111,   Neg. LLF: 14800.002296301875
Iteration:     25,   Func. Count:    136,   Neg. LLF: 14412.205455897278
Iteration:     30,   Func. Count:    161,   Neg. LLF: 14411.399471881272
Iteration:     35,   Func. Count:    186,   Neg. LLF: 14411.391212733033
Iteration:     40,   Func. Count:    211,   Neg. LLF: 14410.33210488145
Iteration:     45,   Func. Count:    236,   Neg. LLF: 14342.017528434071
Iteration:     50,   Func. Count:    266,   Neg. LLF: 14318.223797670367
Iteration:     55,   Func. Count:    296,   Neg. LLF: 14109.992672397046
Iteration:     60,   Func. Count:    321,   Neg. LLF: 14109.522087680747
Optimization terminated successfully    (Exit mode 0)
            Current 

c:\Users\User\anaconda3\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.037e+06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.01 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


